In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.2 MB/s 
     |████████████████████████████████| 81 kB 4.2 MB/s 
     |████████████████████████████████| 209 kB 46.4 MB/s 
     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 50 kB 3.0 MB/s 
     |████████████████████████████████| 147 kB 33.8 MB/s 
     |████████████████████████████████| 112 kB 52.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=6a4ee53b1c2be76d5868e2f4d8d032c250f9b1301f893a9771ac99395e128f83
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
from optuna.samplers import NSGAIISampler
import tensorflow as tf
import optuna
from sklearn.preprocessing import OneHotEncoder

import optuna
from optuna.integration import KerasPruningCallback

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

train_x, test_x = map(lambda x: x[..., None], [train_x, test_x])

encoder = OneHotEncoder(categories=[range(10)])
encoded_train_y, encoded_test_y = map(
    lambda y: encoder.fit_transform(y.reshape(-1, 1)).toarray(), [train_y, test_y]
)

def create_model(trial: optuna.Trial) -> tf.keras.models.Model:
  return tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(train_x.shape[1:]),
        tf.keras.layers.Conv2D(5, (3, 3), activation="relu"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(3, (3, 3), activation="relu"),
        tf.keras.layers.Flatten(),
        *(
            tf.keras.layers.Dense(
                trial.suggest_int(name=f"dense-neurons-{i}", low=8, high=16),
                activation=trial.suggest_categorical(
                    name=f"dense-activation-{i}", choices=["relu", "linear"]
                    ),
                )
                for i in range(trial.suggest_int(name="denses-count", low=1, high=3)
                )
            ),
        tf.keras.layers.Dense(encoded_train_y.shape[-1], activation="softmax"),
    ]
)

def objective(trial: optuna.trial) -> float:
  model = create_model(trial)
  optimizer = tf.keras.optimizers.Adam(learning_rate=10**-5)
  model.compile(
      optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
  )

  model.fit(
      train_x,
      encoded_train_y,
      batch_size=128,
      epochs=5,
      callbacks=[KerasPruningCallback(trial, monitor="val_accuracy")],
      validation_data=(test_x, encoded_test_y),
      validation_batch_size=128,
  )

  optimizer.learning_rate = 10**-3
  model.fit(
      train_x,
      encoded_train_y,
      batch_size=128,
      epochs=5,
      callbacks=[],
      validation_data=(test_x, encoded_test_y),
      validation_batch_size=128,
  )
  loss, accuracy = model.evaluate(test_x, encoded_test_y, batch_size=128)
  return accuracy

study = optuna.create_study(
    study_name="mu-on",
    pruner=optuna.pruners.SuccessiveHalvingPruner(),
    direction=optuna.study.StudyDirection.MAXIMIZE,
    sampler=optuna.samplers.NSGAIISampler()
)

study.optimize(func=objective, n_trials=5, n_jobs=1, show_progress_bar=True)




[I 2022-11-28 10:42:25,055] A new study created in memory with name: mu-on
/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: FutureWarning: KerasPruningCallback has been deprecated in v2.1.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.1.0. Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF: https://github.com/keras-team/keras/releases/tag/2.4.0. There is an alternative callback function that can be used instead: :class:`~optuna.integration.TFKerasPruningCallback`


Epoch 1/5
469/469 [==============================] - 22s 45ms/step - loss: 71.9421 - accuracy: 0.1303 - val_loss: 48.1520 - val_accuracy: 0.1364
Epoch 2/5
469/469 [==============================] - 21s 46ms/step - loss: 31.7958 - accuracy: 0.1578 - val_loss: 18.1198 - val_accuracy: 0.1544
Epoch 3/5
469/469 [==============================] - 21s 44ms/step - loss: 10.9134 - accuracy: 0.1476 - val_loss: 6.2807 - val_accuracy: 0.1311
Epoch 4/5
469/469 [==============================] - 20s 44ms/step - loss: 4.7635 - accuracy: 0.1240 - val_loss: 3.6683 - val_accuracy: 0.1220
Epoch 5/5
469/469 [==============================] - 22s 46ms/step - loss: 3.2631 - accuracy: 0.1248 - val_loss: 2.9094 - val_accuracy: 0.1314
Epoch 1/5
469/469 [==============================] - 21s 44ms/step - loss: 2.1779 - accuracy: 0.1835 - val_loss: 2.0736 - val_accuracy: 0.2227
Epoch 2/5
469/469 [==============================] - 21s 44ms/step - loss: 2.0042 - accuracy: 0.2291 - val_loss: 1.9513 - val_accuracy: 0